https://github.com/VinAIResearch/PhoBERT

https://www.kaggle.com/datasets/gamind/uit-viquad-2-0

In [17]:
# -*- coding: utf-8 -*-
"""
Fine-tuning PhoBERT cho H·ªèi-ƒê√°p Ti·∫øng Vi·ªát v·ªõi UIT-ViQuAD 2.0
"""

# @title 1. C√†i ƒë·∫∑t v√† Import th∆∞ vi·ªán
# C√†i c√°c th∆∞ vi·ªán c·∫ßn thi·∫øt t·ª´ Hugging Face
!pip install transformers datasets accelerate -q

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator # S·∫Ω d√πng ƒë·ªÉ t·∫°o batch d·ªØ li·ªáu
)
from tqdm.auto import tqdm
import collections
import numpy as np

# Ki·ªÉm tra v√† s·ª≠ d·ª•ng GPU n·∫øu c√≥
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thi·∫øt b·ªã ƒëang s·ª≠ d·ª•ng: {device}")

Thi·∫øt b·ªã ƒëang s·ª≠ d·ª•ng: cuda


# Gi·ªõi thi·ªáu Nhanh
M√¥ h√¨nh PhoBERT: L√† m·ªôt m√¥ h√¨nh Transformer ƒë∆∞·ª£c VinAI Research hu·∫•n luy·ªán tr∆∞·ªõc (pre-trained) ƒë·∫∑c bi·ªát cho ti·∫øng Vi·ªát. N√≥ c√≥ kh·∫£ nƒÉng hi·ªÉu ng·ªØ c·∫£nh v√† ng·ªØ nghƒ©a ti·∫øng Vi·ªát t·ªët, ph√π h·ª£p cho nhi·ªÅu t√°c v·ª• NLP, bao g·ªìm H·ªèi-ƒê√°p. Ch√∫ng ta s·∫Ω s·ª≠ d·ª•ng phi√™n b·∫£n vinai/phobert-base.

D·ªØ li·ªáu UIT-ViQuAD 2.0: L√† b·ªô d·ªØ li·ªáu H·ªèi-ƒê√°p Ti·∫øng Vi·ªát do ƒê·∫°i h·ªçc CNTT (UIT) ph√°t tri·ªÉn, d·ª±a tr√™n ƒë·ªãnh d·∫°ng SQuAD 2.0. N√≥ ch·ª©a c√°c c·∫∑p (ng·ªØ c·∫£nh, c√¢u h·ªèi, c√¢u tr·∫£ l·ªùi) l·∫•y t·ª´ Wikipedia ti·∫øng Vi·ªát. ƒêi·ªÉm ƒë·∫∑c bi·ªát l√† b·ªô d·ªØ li·ªáu n√†y c√≥ c·∫£ nh·ªØng c√¢u h·ªèi c√≥ th·ªÉ tr·∫£ l·ªùi ƒë∆∞·ª£c v√† kh√¥ng th·ªÉ tr·∫£ l·ªùi ƒë∆∞·ª£c t·ª´ ng·ªØ c·∫£nh cho tr∆∞·ªõc.

In [18]:
# @title 2. T·∫£i D·ªØ li·ªáu UIT-ViQuAD 2.0
# T·∫£i dataset t·ª´ Hugging Face Hub
dataset_name = "taidng/UIT-ViQuAD2.0"
try:
    raw_datasets = load_dataset(dataset_name)
    print("\nT·∫£i dataset th√†nh c√¥ng:")
    print(raw_datasets)
    # In th·ª≠ m·ªôt m·∫´u
    print("\nM·∫´u d·ªØ li·ªáu (train):")
    print(raw_datasets["train"][10]) # In m·∫´u th·ª© 10
except Exception as e:
    print(f"L·ªói khi t·∫£i dataset: {e}")
    print("Vui l√≤ng ki·ªÉm tra l·∫°i t√™n dataset ho·∫∑c k·∫øt n·ªëi m·∫°ng.")


T·∫£i dataset th√†nh c√¥ng:
DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 28454
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

M·∫´u d·ªØ li·ªáu (train):
{'id': '0001-0002-0004', 'uit_id': 'uit_000011', 'title': 'Ph·∫°m VƒÉn ƒê·ªìng', 'context': 'NƒÉm 1954, √¥ng ƒë∆∞·ª£c giao nhi·ªám v·ª• Tr∆∞·ªüng ph√°i ƒëo√†n Ch√≠nh ph·ªß d·ª± H·ªôi ngh·ªã Gen√®ve v·ªÅ ƒê√¥ng D∆∞∆°ng. Nh·ªØng ƒë√≥ng g√≥p c·ªßa ƒëo√†n Vi·ªát Nam do √¥ng ƒë·ª©ng ƒë·∫ßu l√† v√¥ c√πng quan tr·ªçng, t·∫°o ra nh·ªØng ƒë·ªôt ph√° ƒë∆∞a H·ªôi ngh·ªã t·ªõi th√†nh c√¥ng. Tr·∫£i qua 8 phi√™n h·ªçp to√†n th·

In [23]:
# @title 3. T·∫£i Model PhoBERT v√† Tokenizer
# T·∫£i tokenizer v√† model PhoBERT ƒë√£ pre-trained cho t√°c v·ª• Question Answering
# model_checkpoint = "vinai/phobert-base"

# @title 3. T·∫£i Model XLM-RoBERTa v√† Tokenizer
# T·∫£i tokenizer v√† model XLM-RoBERTa ƒë√£ pre-trained cho t√°c v·ª• Question Answering
model_checkpoint = "xlm-roberta-base" # <<<=== THAY ƒê·ªîI ·ªû ƒê√ÇY
try:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True) # V·∫´n gi·ªØ use_fast=True
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to(device)
    print(f"\nT·∫£i model {model_checkpoint} v√† tokenizer th√†nh c√¥ng.")

    # === TH√äM 2 D√íNG N√ÄY ƒê·ªÇ KI·ªÇM TRA ===
    print(f"Lo·∫°i Tokenizer: {type(tokenizer)}")
    print(f"L√† Fast Tokenizer: {tokenizer.is_fast}")
    # ==================================

except Exception as e:
    print(f"L·ªói khi t·∫£i model ho·∫∑c tokenizer: {e}")
    print("Vui l√≤ng ki·ªÉm tra l·∫°i t√™n model checkpoint.")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



T·∫£i model xlm-roberta-base v√† tokenizer th√†nh c√¥ng.
Lo·∫°i Tokenizer: <class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
L√† Fast Tokenizer: True


In [26]:
# @title 4. Ti·ªÅn x·ª≠ l√Ω D·ªØ li·ªáu (Quan tr·ªçng) - PHI√äN B·∫¢N ƒê√É S·ª¨A L·ªñI

# C√°c tham s·ªë cho vi·ªác ti·ªÅn x·ª≠ l√Ω
max_length = 384
doc_stride = 128

def prepare_features(examples):
    # Th√™m kho·∫£ng tr·∫Øng tr∆∞·ªõc c√¢u h·ªèi
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize context v√† question
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Th√™m c√°c tr∆∞·ªùng label m·ªõi
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # Ki·ªÉm tra xem tr∆∞·ªùng 'answers' c√≥ t·ªìn t·∫°i trong batch n√†y kh√¥ng
    # Gi·∫£ ƒë·ªãnh r·∫±ng n·∫øu kh√¥ng c√≥ 'answers' th√¨ ƒë√≥ l√† test set
    answers_exist = "answers" in examples and examples["answers"] is not None

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]

        # ƒê·∫∑t gi√° tr·ªã m·∫∑c ƒë·ªãnh (cho test set ho·∫∑c c√¢u h·ªèi kh√¥ng tr·∫£ l·ªùi ƒë∆∞·ª£c)
        start_position = cls_index
        end_position = cls_index

        # Ch·ªâ x·ª≠ l√Ω t√¨m v·ªã tr√≠ start/end n·∫øu 'answers' t·ªìn t·∫°i
        if answers_exist and sample_index < len(examples["answers"]): # Th√™m ki·ªÉm tra sample_index
            answers = examples["answers"][sample_index]
             # Ki·ªÉm tra xem 'is_impossible' c√≥ t·ªìn t·∫°i kh√¥ng (c√≥ th·ªÉ c√≥ ho·∫∑c kh√¥ng trong test set th·∫≠t)
            is_impossible = examples.get("is_impossible", [True] * len(examples["context"]))[sample_index] # L·∫•y flag, m·∫∑c ƒë·ªãnh True n·∫øu thi·∫øu

            # Ch·ªâ t√¨m v·ªã tr√≠ n·∫øu c√¢u h·ªèi c√≥ th·ªÉ tr·∫£ l·ªùi ƒë∆∞·ª£c v√† c√≥ c√¢u tr·∫£ l·ªùi
            if answers and not is_impossible and len(answers["answer_start"]) > 0:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Ki·ªÉm tra c√¢u tr·∫£ l·ªùi c√≥ n·∫±m trong span kh√¥ng
                if offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char:
                    # T√¨m token start
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    start_position = token_start_index - 1
                    # T√¨m token end
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    end_position = token_end_index + 1
                # else: kh√¥ng n·∫±m trong span, gi·ªØ nguy√™n start/end l√† cls_index

        # L∆∞u k·∫øt qu·∫£ v·ªã tr√≠ start/end
        tokenized_examples["start_positions"].append(start_position)
        tokenized_examples["end_positions"].append(end_position)

    return tokenized_examples


# √Åp d·ª•ng h√†m ti·ªÅn x·ª≠ l√Ω v√†o to√†n b·ªô dataset
print("\nB·∫Øt ƒë·∫ßu ti·ªÅn x·ª≠ l√Ω dataset...")
try:
    # X√≥a c·ªôt 'answers' v√† 'is_impossible' tr∆∞·ªõc khi map n·∫øu ch√∫ng g√¢y l·ªói (ch·ªâ √°p d·ª•ng cho test set n·∫øu c√≥)
    # Tuy nhi√™n, c√°ch t·ªët h∆°n l√† x·ª≠ l√Ω trong h√†m map nh∆∞ tr√™n
    tokenized_datasets = raw_datasets.map(
        prepare_features,
        batched=True,
        remove_columns=raw_datasets["train"].column_names # V·∫´n x√≥a c·ªôt c≈© ƒë·ªÉ g·ªçn g√†ng
    )
    print("\nTi·ªÅn x·ª≠ l√Ω ho√†n t·∫•t.")
    print("Dataset sau ti·ªÅn x·ª≠ l√Ω:")
    print(tokenized_datasets)
except Exception as e:
    print(f"\nL·ªói trong qu√° tr√¨nh ti·ªÅn x·ª≠ l√Ω: {e}")


B·∫Øt ƒë·∫ßu ti·ªÅn x·ª≠ l√Ω dataset...


Map:   0%|          | 0/28454 [00:00<?, ? examples/s]

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

Map:   0%|          | 0/7301 [00:00<?, ? examples/s]


Ti·ªÅn x·ª≠ l√Ω ho√†n t·∫•t.
Dataset sau ti·ªÅn x·ª≠ l√Ω:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 30399
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3937
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 7695
    })
})


In [27]:
# @title 5. Hu·∫•n luy·ªán M√¥ h√¨nh (Fine-tuning)

# ƒê·ªãnh nghƒ©a n∆°i l∆∞u model v√† c√°c tham s·ªë hu·∫•n luy·ªán
output_dir = "phobert-finetuned-viquad2"
# Gi·∫£m batch_size n·∫øu g·∫∑p l·ªói Out-of-Memory (OOM) tr√™n Colab
# TƒÉng s·ªë epochs n·∫øu mu·ªën hu·∫•n luy·ªán k·ªπ h∆°n (nh∆∞ng t·ªën th·ªùi gian h∆°n)
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch", # ƒê√°nh gi√° sau m·ªói epoch
    learning_rate=3e-5,          # Learning rate ph·ªï bi·∫øn cho fine-tuning BERT
    per_device_train_batch_size=8, # Gi·∫£m n·∫øu c·∫ßn (vd: 4)
    per_device_eval_batch_size=8,  # Gi·∫£m n·∫øu c·∫ßn
    num_train_epochs=2,          # S·ªë epoch hu·∫•n luy·ªán (tƒÉng l√™n 3 ho·∫∑c h∆°n n·∫øu c·∫ßn)
    weight_decay=0.01,
    save_strategy="epoch",       # L∆∞u model sau m·ªói epoch
    load_best_model_at_end=True, # T·∫£i model t·ªët nh·∫•t sau khi hu·∫•n luy·ªán xong
    # push_to_hub=False,         # ƒê·∫∑t True n·∫øu mu·ªën ƒë·∫©y l√™n Hugging Face Hub
)

# Kh·ªüi t·∫°o Trainer
# default_data_collator s·∫Ω t·ª± ƒë·ªông t·∫°o batch t·ª´ dataset ƒë√£ x·ª≠ l√Ω
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # S·ª≠ d·ª•ng t·∫≠p validation ƒë·ªÉ ƒë√°nh gi√°
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# B·∫Øt ƒë·∫ßu hu·∫•n luy·ªán
# Qu√° tr√¨nh n√†y c√≥ th·ªÉ m·∫•t th·ªùi gian ƒë√°ng k·ªÉ, ph·ª• thu·ªôc v√†o GPU c·ªßa Colab
print("\nB·∫Øt ƒë·∫ßu qu√° tr√¨nh fine-tuning...")
try:
    trainer.train()
    print("\nHu·∫•n luy·ªán ho√†n t·∫•t!")
except Exception as e:
    print(f"\nL·ªói trong qu√° tr√¨nh hu·∫•n luy·ªán: {e}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-27-c2305ad013ba>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



B·∫Øt ƒë·∫ßu qu√° tr√¨nh fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anime365247-aaa (anime365247-aaa-tuy-n-sinh-h-spkt-tphcm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Hu·∫•n luy·ªán ho√†n t·∫•t!


In [28]:
# @title 6. L∆∞u Model ƒë√£ Hu·∫•n luy·ªán
# L∆∞u model v√† tokenizer t·ªët nh·∫•t v√†o th∆∞ m·ª•c output_dir
print(f"\nL∆∞u model v√† tokenizer v√†o th∆∞ m·ª•c: {output_dir}")
try:
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("L∆∞u th√†nh c√¥ng!")
    print(f"B·∫°n c√≥ th·ªÉ t√¨m th·∫•y model trong th∆∞ m·ª•c '{output_dir}' ·ªü c√¢y th∆∞ m·ª•c b√™n tr√°i c·ªßa Colab.")
except Exception as e:
    print(f"L·ªói khi l∆∞u model: {e}")


# @title 7. (T√πy ch·ªçn) S·ª≠ d·ª•ng Model ƒë·ªÉ D·ª± ƒëo√°n
# V√≠ d·ª• c√°ch d√πng model ƒë√£ fine-tune ƒë·ªÉ tr·∫£ l·ªùi c√¢u h·ªèi

from transformers import pipeline

# T·∫£i model ƒë√£ l∆∞u (n·∫øu c·∫ßn ch·∫°y l·∫°i session)
# saved_model_path = output_dir
# qa_pipeline = pipeline("question-answering", model=saved_model_path, tokenizer=saved_model_path, device=0 if torch.cuda.is_available() else -1)

# Ho·∫∑c s·ª≠ d·ª•ng model ƒëang c√≥ trong b·ªô nh·ªõ (n·∫øu v·ª´a hu·∫•n luy·ªán xong)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


# V√≠ d·ª•
context = "Tr∆∞·ªùng ƒê·∫°i h·ªçc C√¥ng ngh·ªá Th√¥ng tin (UIT) l√† m·ªôt tr∆∞·ªùng ƒë·∫°i h·ªçc th√†nh vi√™n c·ªßa ƒê·∫°i h·ªçc Qu·ªëc gia Th√†nh ph·ªë H·ªì Ch√≠ Minh, ƒë∆∞·ª£c th√†nh l·∫≠p v√†o ng√†y 8 th√°ng 6 nƒÉm 2006. Tr∆∞·ªùng chuy√™n ƒë√†o t·∫°o v·ªÅ lƒ©nh v·ª±c c√¥ng ngh·ªá th√¥ng tin v√† truy·ªÅn th√¥ng."
question = "UIT ƒë∆∞·ª£c th√†nh l·∫≠p khi n√†o?"

print("\nV√≠ d·ª• d·ª± ƒëo√°n:")
print(f"Context: {context}")
print(f"Question: {question}")

try:
    result = qa_pipeline(question=question, context=context)
    print("\nK·∫øt qu·∫£ d·ª± ƒëo√°n:")
    print(f"Answer: {result['answer']}")
    print(f"Score: {result['score']:.4f}")
    print(f"Start: {result['start']}")
    print(f"End: {result['end']}")
except Exception as e:
     print(f"\nL·ªói khi d·ª± ƒëo√°n: {e}")

# V√≠ d·ª• c√¢u h·ªèi kh√¥ng c√≥ c√¢u tr·∫£ l·ªùi trong context
question_impossible = "Ai l√† hi·ªáu tr∆∞·ªüng ƒë·∫ßu ti√™n c·ªßa UIT?"
print(f"\nQuestion (Impossible): {question_impossible}")
try:
    result_impossible = qa_pipeline(question=question_impossible, context=context)
    print("\nK·∫øt qu·∫£ d·ª± ƒëo√°n (Impossible):")
     # Model t·ªët s·∫Ω tr·∫£ v·ªÅ score th·∫•p ho·∫∑c chu·ªói r·ªóng (t√πy c√°ch x·ª≠ l√Ω)
    print(f"Answer: {result_impossible['answer']}")
    print(f"Score: {result_impossible['score']:.4f}")
except Exception as e:
     print(f"\nL·ªói khi d·ª± ƒëo√°n: {e}")


L∆∞u model v√† tokenizer v√†o th∆∞ m·ª•c: phobert-finetuned-viquad2


Device set to use cuda:0


L∆∞u th√†nh c√¥ng!
B·∫°n c√≥ th·ªÉ t√¨m th·∫•y model trong th∆∞ m·ª•c 'phobert-finetuned-viquad2' ·ªü c√¢y th∆∞ m·ª•c b√™n tr√°i c·ªßa Colab.

V√≠ d·ª• d·ª± ƒëo√°n:
Context: Tr∆∞·ªùng ƒê·∫°i h·ªçc C√¥ng ngh·ªá Th√¥ng tin (UIT) l√† m·ªôt tr∆∞·ªùng ƒë·∫°i h·ªçc th√†nh vi√™n c·ªßa ƒê·∫°i h·ªçc Qu·ªëc gia Th√†nh ph·ªë H·ªì Ch√≠ Minh, ƒë∆∞·ª£c th√†nh l·∫≠p v√†o ng√†y 8 th√°ng 6 nƒÉm 2006. Tr∆∞·ªùng chuy√™n ƒë√†o t·∫°o v·ªÅ lƒ©nh v·ª±c c√¥ng ngh·ªá th√¥ng tin v√† truy·ªÅn th√¥ng.
Question: UIT ƒë∆∞·ª£c th√†nh l·∫≠p khi n√†o?

K·∫øt qu·∫£ d·ª± ƒëo√°n:
Answer: ng√†y 8 th√°ng 6 nƒÉm 2006.
Score: 0.6327
Start: 137
End: 161

Question (Impossible): Ai l√† hi·ªáu tr∆∞·ªüng ƒë·∫ßu ti√™n c·ªßa UIT?

K·∫øt qu·∫£ d·ª± ƒëo√°n (Impossible):
Answer: ƒê·∫°i h·ªçc Qu·ªëc gia Th√†nh ph·ªë H·ªì Ch√≠ Minh,
Score: 0.0576


In [29]:
# @title 7. ƒê√≥ng g√≥i v√† T·∫£i Model (Zip and Download Model)

import shutil
from google.colab import files
import os

# --- L·∫•y t√™n th∆∞ m·ª•c ƒë√£ l∆∞u model ---
# ƒê·∫£m b·∫£o bi·∫øn `output_dir` ƒë√£ ƒë∆∞·ª£c ƒë·ªãnh nghƒ©a ·ªü Cell 5 v√† ch·ª©a ƒë√∫ng t√™n th∆∞ m·ª•c
try:
    # T√™n th∆∞ m·ª•c ch·ª©a model ƒë√£ l∆∞u (v√≠ d·ª•: "phobert-finetuned-viquad2" ho·∫∑c "xlm-roberta-finetuned-viquad2")
    saved_model_directory = output_dir

    print(f"[*] Th∆∞ m·ª•c ch·ª©a model c·∫ßn ƒë√≥ng g√≥i: '{saved_model_directory}'")

    # Ki·ªÉm tra xem th∆∞ m·ª•c c√≥ th·ª±c s·ª± t·ªìn t·∫°i kh√¥ng
    if os.path.isdir(saved_model_directory):
        # --- T·∫°o t√™n file zip (gi·ªëng t√™n th∆∞ m·ª•c) ---
        zip_filename_base = saved_model_directory
        zip_filepath = f"{zip_filename_base}.zip" # T√™n file zip cu·ªëi c√πng

        print(f"[*] ƒêang n√©n th∆∞ m·ª•c '{saved_model_directory}' th√†nh file '{zip_filepath}'...")

        # --- Th·ª±c hi·ªán n√©n th∆∞ m·ª•c ---
        # C√∫ ph√°p: shutil.make_archive(t√™n_file_n√©n_kh√¥ng_ƒëu√¥i, 'ƒë·ªãnh_d·∫°ng', th∆∞_m·ª•c_g·ªëc_c·∫ßn_n√©n)
        shutil.make_archive(zip_filename_base, 'zip', saved_model_directory)

        print(f"[*] N√©n th√†nh c√¥ng! File '{zip_filepath}' ƒë√£ ƒë∆∞·ª£c t·∫°o.")

        # --- K√≠ch ho·∫°t t·∫£i file zip v·ªÅ m√°y t√≠nh ---
        print(f"[*] Chu·∫©n b·ªã t·∫£i file '{zip_filepath}' v·ªÅ m√°y...")
        files.download(zip_filepath)
        print("[*] Ho√†n t·∫•t! Ki·ªÉm tra c·ª≠a s·ªï download c·ªßa tr√¨nh duy·ªát.")

    else:
        print(f"[!] L·ªói: Kh√¥ng t√¨m th·∫•y th∆∞ m·ª•c '{saved_model_directory}'.")
        print("    H√£y ch·∫Øc ch·∫Øn r·∫±ng b·∫°n ƒë√£ ch·∫°y Cell 6 (L∆∞u Model) th√†nh c√¥ng.")

except NameError:
    # X·ª≠ l√Ω tr∆∞·ªùng h·ª£p bi·∫øn output_dir ch∆∞a ƒë∆∞·ª£c t·∫°o
    print("[!] L·ªói: Kh√¥ng t√¨m th·∫•y bi·∫øn 'output_dir'.")
    print("    Vui l√≤ng ki·ªÉm tra l·∫°i Cell 5 n∆°i b·∫°n ƒë·ªãnh nghƒ©a `TrainingArguments` v√† ƒë·∫£m b·∫£o n√≥ ƒë√£ ch·∫°y.")
except Exception as e:
    # B·∫Øt c√°c l·ªói kh√°c c√≥ th·ªÉ x·∫£y ra
    print(f"[!] C√≥ l·ªói x·∫£y ra trong qu√° tr√¨nh ƒë√≥ng g√≥i ho·∫∑c t·∫£i v·ªÅ: {e}")

[*] Th∆∞ m·ª•c ch·ª©a model c·∫ßn ƒë√≥ng g√≥i: 'phobert-finetuned-viquad2'
[*] ƒêang n√©n th∆∞ m·ª•c 'phobert-finetuned-viquad2' th√†nh file 'phobert-finetuned-viquad2.zip'...
[*] N√©n th√†nh c√¥ng! File 'phobert-finetuned-viquad2.zip' ƒë√£ ƒë∆∞·ª£c t·∫°o.
[*] Chu·∫©n b·ªã t·∫£i file 'phobert-finetuned-viquad2.zip' v·ªÅ m√°y...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[*] Ho√†n t·∫•t! Ki·ªÉm tra c·ª≠a s·ªï download c·ªßa tr√¨nh duy·ªát.
